In [3]:
import os
import pickle
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error

#sys.path.append(str(Path(__file__).absolute().parents[1]))

os.system('cls' if os.name == 'nt' else 'clear')

# Size of the dataset
LAST_INDEX_TRAIN = -1  # Tudo = -1
LAST_INDEX_TEST = -1  # Tudo = -1

TRAIN_PERCENTAGE = 0.8
COLUMNS_TO_REMOVE = ['Pacientes','Nome']
TARGET_NAME = 'Diagnosticounificado'

# Diagnosticounificado;T-Score L1-L4;T-Score Colo Femoral;T-Score Fêmur Total

# Define the CSVs path 
FILE_PATH = 'train_database_matheus.csv'
PLOT_PATH = '/d01/scholles/gigasistemica/matheus-giga/plots/regression/regression.png'

# Define where the trained model is saved
PATH_TRAINED_MODEL = '/d01/scholles/gigasistemica/radiomics/radiomics_rf_model.sav'

def read_csv_and_split(path, separation_percentage, columns_to_remove):
    # Read the CSV
    df = pd.read_csv(path, sep=';')

    # Calculate the number of rows to be separated
    num_rows_to_split = int(len(df) * separation_percentage)
    
    # Generate random indices for separation
    split_indices = np.random.choice(df.index, size=num_rows_to_split, replace=False)
    
    # Create separated DataFrames
    df_train = df.loc[split_indices]
    df_test = df.drop(split_indices)
    
    # Remove the specified columns
    df_train = df_train.drop(columns=columns_to_remove, errors='ignore')
    df_test = df_test.drop(columns=columns_to_remove, errors='ignore')
    
    # Get the list of remaining columns
    all_columns = list(df_test.columns)
    
    return df_train, df_test, all_columns

if __name__ == '__main__':

    train_df, df_test, all_columns = read_csv_and_split(FILE_PATH, TRAIN_PERCENTAGE, COLUMNS_TO_REMOVE)
    features = ['Altura da cortical (D)','Altura forame-base (D)','Altura da medular forame-superficie  da cortical (D)','Altura da cortical (E)','Altura forame-base (E)','Altura da medular forame-superficie  da cortical (E)']
    target = [TARGET_NAME]    
    
    X = train_df[features].to_numpy()
    y = train_df[target].to_numpy()

    # Specifies the learning model and performs the training
    learning_model = RandomForestClassifier(verbose=True)
    print('Iniciado o treino!')
    learning_model.fit(X, y.ravel())

    
    # -------------------------------------------------------------------------------------------------------------#
    # ------------------ Validation Code for statistics ------------------#
    # -------------------------------------------------------------------------------------------------------------#

    print('Iniciada predição!')
    x_pred = df_test[features].to_numpy()
    y_pred = learning_model.predict(x_pred)
    y_true = df_test[target].to_numpy()

    print('\n\n',classification_report(y_true, y_pred, digits=4))

Iniciado o treino!
Iniciada predição!


               precision    recall  f1-score   support

           0     0.2857    0.2000    0.2353        10
           1     0.3846    0.5000    0.4348        10

    accuracy                         0.3500        20
   macro avg     0.3352    0.3500    0.3350        20
weighted avg     0.3352    0.3500    0.3350        20



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


Only choosing some specific feature,probably from expert knowledge didn't produce good results. We can see accuracy is around only 30 percent.So, we have to do another way for classification.